# Day 1

https://adventofcode.com/2021/day/1

In [1]:
with open("./input/1") as file:
    depths = [int(line) for line in file]

## Part 1

In [2]:
sum(int(d2 > d1) for d1, d2 in zip(depths[:-1], depths[1:]))

1477

## Part 2

In [3]:
sum(int(d4 > d1) for d1, d4 in zip(depths[:-3], depths[3:]))

1523

# Day 2

https://adventofcode.com/2021/day/2

In [4]:
from numpy import array  # I'm lazy

In [5]:
def parse_instruction(instruction):
    direction, _distance = instruction.split()
    distance = int(_distance)
    if direction == "forward":
        return (distance, 0)
    return (0, (1 if direction == "down" else -1) * distance)

In [6]:
with open("./input/2") as file:
    instructions = array([parse_instruction(line) for line in file])

## Part 1

In [7]:
forward_distance, depth = instructions.sum(axis=0)

In [8]:
forward_distance * depth

1936494

## Part 2

In [9]:
forward_instr, aim_instr = instructions.T

In [10]:
sum(forward_instr) * (forward_instr * aim_instr.cumsum()).sum()

1997106066

# Day 3

https://adventofcode.com/2021/day/3

In [11]:
from numpy import array

In [12]:
with open("./input/3") as file:
    bit_list = array([[int(bit) for bit in line[:-1]] for line in file])

In [13]:
def bits_to_int(bits):
    return int("".join([str(bit) for bit in bits]), base=2)

## Part 1

In [14]:
gamma = bits_to_int(["1" if avg > 0.5 else "0" for avg in bit_list.mean(axis=0)])
epsilon = bits_to_int(["0" if avg > 0.5 else "1" for avg in bit_list.mean(axis=0)])

In [15]:
gamma * epsilon

4006064

## Part 2

In [16]:
def _filter_bits(bit_list, position, gas="o2"):
    most_common_bit = int(bit_list[:, position].mean(axis=0) + 0.5)
    if gas == "o2":
        return bit_list[[bits[position] == most_common_bit for bits in bit_list]]
    if gas == "co2":
        return bit_list[[bits[position] == 1 - most_common_bit for bits in bit_list]]
    raise ValueError(f"Expected o2 or co2 gas, got {gas}.")

In [17]:
def reduce_bits(bit_list, gas="o2"):
    current_list = bit_list
    for position in range(bit_list.shape[1]):
        current_list = _filter_bits(current_list, position, gas)
        if len(current_list) == 1:
            return current_list[0]
    raise RuntimeError("Did not find a unique result")

In [18]:
bits_to_int(reduce_bits(bit_list, "o2")) * bits_to_int(reduce_bits(bit_list, "co2"))

5941884

# Day 4

https://adventofcode.com/2021/day/4 (for another type of games involving a squid, see [here](https://en.wikipedia.org/wiki/Squid_Game))

In [19]:
from numpy import array

In [20]:
with open("./input/4") as file:
    calls = [int(num) for num in next(file).split(",")]
    _ = next(file)  # second line is empty
    boards = []
    _board = []
    for line in file:
        if line == "\n":
            boards.append(_board)
            _board = []
            continue
        _board.append([int(s) for s in line.split()])

In [21]:
def call_number(board, number):
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == number:
                return i, j

In [22]:
def has_won(board_calls, board_height, board_width):
    full_rows = [[(i, j) for j in range(board_width)] for i in range(board_height)]
    full_cols = [[(i, j) for i in range(board_height)] for j in range(board_width)]
    return (any(all(coord in board_calls for coord in full_row) for full_row in full_rows)
            or any(all(coord in board_calls for coord in full_col) for full_col in full_cols))

## Part 1

In [23]:
def bingo(boards, calls):
    state = {i: [] for i in range(len(boards))}
    for call in calls:
        for i, board in enumerate(boards):
            call_position = call_number(board, call)
            if call_position:
                board_state = state[i]
                board_state.append(call_position)
                if has_won(board_state, len(board), len(board[1])):  # B I N G O
                    unmarked = sum(board[i][j] for i in range(len(board)) for j in range(len(board[i]))
                                   if (i, j) not in board_state)
                    return unmarked * call

In [24]:
bingo(boards, calls)

10680

## Part 2

In [25]:
def losing_bingo(boards, calls):
    state = {i: [] for i in range(len(boards))}
    remaining_boards = set(range(len(boards)))
    for call in calls:
        for i, board in enumerate(boards):
            if i not in remaining_boards:
                continue
            call_position = call_number(board, call)
            if call_position:
                board_state = state[i]
                board_state.append(call_position)
                if has_won(board_state, len(board), len(board[1])):
                    if len(remaining_boards) > 1:
                        remaining_boards.remove(i)
                        continue
                    unmarked = sum(board[i][j] for i in range(len(board)) for j in range(len(board[i]))
                                   if (i, j) not in board_state)
                    return unmarked * call

In [26]:
losing_bingo(boards, calls)

31892

# Day 5

https://adventofcode.com/2021/day/5

In [27]:
from collections import defaultdict

In [28]:
def parse_coord(coord):
    x, y = coord.split(",")
    return int(x), int(y)

In [29]:
with open("./input/5") as file:
    lines = [[parse_coord(start), parse_coord(end)]
             for start, end in [line.split(" -> ") for line in file.read().splitlines()]]

In [30]:
def points_in_line(start, end):
    (start_x, start_y), (end_x, end_y) = sorted([start, end])
    if start_x == end_x:  # vertical
        return [(start_x, y) for y in range(start_y, end_y + 1)]
    if start_y == end_y:  # horizontal
        return [(x, start_y) for x in range(start_x, end_x + 1)]
    if start_y < end_y:  # "upwards" diagonal
        return [(start_x + i, start_y + i) for i in range(end_x - start_x + 1)]
    # "downwards" diagonal
    return [(start_x + i, start_y - i) for i in range(end_x - start_x + 1)]

## Part 1

In [31]:
def fill_grid_no_diagonals():
    grid = defaultdict(lambda: 0)
    for start, end in lines:
        if start[0] != end[0] and start[1] != end[1]:
            continue
        for point in points_in_line(start, end):
            grid[point] += 1
    return grid

In [32]:
sum(1 for point, overlap in fill_grid_no_diagonals().items() if overlap > 1)

4993

## Part 2

In [33]:
def fill_grid():
    grid = defaultdict(lambda: 0)
    for start, end in lines:
        for point in points_in_line(start, end):
            grid[point] += 1
    return grid

In [34]:
sum(1 for point, overlap in fill_grid().items() if overlap > 1)

21101

# Day 6

https://adventofcode.com/2021/day/6

In [35]:
with open("./input/6") as file:
    timers = [int(timer) for timer in file.read().split(",")]

In [36]:
count_by_timer = {timer: 0 for timer in range(9)}
for timer in timers:
    count_by_timer[timer] += 1

In [37]:
def update_count(count_by_timer):
    new_count = {k - 1: v for k, v in count_by_timer.items() if k > 0}
    zero_count = count_by_timer[0]
    new_count[8] = zero_count
    new_count[6] += zero_count
    return new_count

In [38]:
def run(count_by_timer, nturns):
    for _ in range(nturns):
        count_by_timer = update_count(count_by_timer)
    return count_by_timer

## Part 1

In [39]:
sum(run(count_by_timer, 80).values())

390011

## Part 2

In [40]:
sum(run(count_by_timer, 256).values())

1746710169834

# Day 7

https://adventofcode.com/2021/day/7

In [41]:
with open("./input/7") as file:
    positions = [int(p) for p in file.read().split(",")]

## Part 1

In [42]:
min(sum(abs(position - n) for position in positions) for n in range(max(positions)))

347449

## Part 2

In [43]:
def fuel(diff):
    return int(diff * (diff + 1) / 2)

In [44]:
min(sum(fuel(abs(position - n)) for position in positions) for n in range(max(positions)))

98039527

# Day 8

https://adventofcode.com/2021/day/8

In [45]:
with open("./input/8") as file:
    patterns, numbers = zip(*[[string.split() for string in line.split(" | ")]
                              for line in file.read().splitlines()])

## Part 1

In [46]:
sum(1 for number in numbers for digit in number if len(digit) in [2, 3, 4, 7])

321

## Part 2

In [47]:
proper_display = {
    'cf': 1,
    'acf': 7,
    'bcdf': 4,
    'acdeg': 2,
    'acdfg': 3,
    'abdfg': 5,
    'abcefg': 0,
    'abdefg': 6,
    'abcdfg': 9,
    'abcdefg': 8,
}

In [48]:
from itertools import permutations

In [49]:
def shuffle(number, permutation):
    return "".join(sorted("abcdefg"[permutation["abcdefg".index(char)]] for char in number))

In [50]:
true_numbers = set(proper_display)

In [51]:
decoded_list = []
for pattern, number in zip(patterns, numbers):
    for permutation in permutations(range(7)):
        if set(shuffle(pattern_digit, permutation) for pattern_digit in pattern) == true_numbers:
            decoded_list.append(sum(10 ** (len(number) - i - 1) * proper_display[shuffle(digit, permutation)]
                                    for i, digit in enumerate(number)))
            break

In [52]:
sum(decoded_list)

1028926

# Day 9

https://adventofcode.com/2021/day/9

In [53]:
with open("./input/9") as file:
    heights = [[int(c) for c in line] for line in file.read().splitlines()]

In [54]:
def get_neighbours(i, j, heights):
    return [
        neighbour for neighbour in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
        if 0 <= neighbour[0] < len(heights) and 0 <= neighbour[1] < len(heights[i])
    ]

## Part 1

In [55]:
low_points = []
for i in range(len(heights)):
    for j in range(len(heights[i])):
        neighbours = get_neighbours(i, j, heights)
        if all(heights[i][j] < heights[neighbour[0]][neighbour[1]]
               for neighbour in neighbours):
            low_points.append((i, j))

In [56]:
sum(heights[i][j] for i, j in low_points) + len(low_points)

423

## Part 2

In [57]:
def get_basins(heights):
    basins = {}
    unexplored = {(i, j) for i in range(len(heights)) for j in range(len(heights[i]))
                  if heights[i][j] < 9}
    while unexplored:
        point = unexplored.pop()
        non_peak_neighbours = {(i, j) for (i, j) in get_neighbours(*point, heights) if heights[i][j] < 9}

        touched_basins = [(basin_center, basin) for basin_center, basin in basins.items()
                          if any(neighbour in basin for neighbour in non_peak_neighbours)]
        
        if not touched_basins:
            basins[point] = {point} | non_peak_neighbours
            continue

        main_basin, *other_basins = touched_basins

        main_basin[1].update({point} | non_peak_neighbours)

        for other_basin_center, other_basin in other_basins:
            main_basin[1].update(other_basin)
            del basins[other_basin_center]
    return basins

In [58]:
from functools import reduce
from operator import mul

In [59]:
reduce(mul, sorted([len(basin) for basin in get_basins(heights).values()])[-3:], 1)

1198704

# Day 10

https://adventofcode.com/2021/day/10

In [60]:
with open("./input/10") as file:
    lines = file.read().splitlines()

In [61]:
chars = {
    "(": ")",
    "[": "]",
    "{": "}",
    "<": ">",
}

In [62]:
def first_invalid(line):
    pile = []
    for char in line:
        if char in chars:
            pile.append(char)
            continue
        if not pile or char != chars[pile[-1]]:
            return char
        pile.pop()

## Part 1

In [63]:
error_scores = {
    ")": 3,
    "]": 57,
    "}": 1197,
    ">": 25137,
}

In [64]:
sum(error_scores.get(first_invalid(line), 0) for line in lines)

243939

## Part 2

In [65]:
autocomplete_scores = {
    ")": 1,
    "]": 2,
    "}": 3,
    ">": 4,
}

In [66]:
def fix_line(line):
    pile = []
    for char in line:
        if char in chars:
            pile.append(char)
            continue
        last_open = pile.pop()
        assert char == chars[last_open]
    return [chars[open_char] for open_char in pile[::-1]]

In [67]:
def get_score(fix):
    score = 0
    for char in fix:
        score = score * 5 + autocomplete_scores[char]
    return score

In [68]:
incomplete_lines = [line for line in lines if not first_invalid(line)]

In [69]:
sorted(get_score(fix_line(line)) for line in incomplete_lines)[(len(incomplete_lines) - 1) // 2]

2421222841